In [2]:
# The code was removed by Watson Studio for sharing.

,Date,BTC Open,BTC Close,BTC Volume,TESLA Open,TESLA Close,TESLA Volume,NASDAQ Open,NASDAQ Close,NASDAQ Volume,Disaster,Day of Week,day
0,2021-12-31,47207.99,46806.83,1591.712555,1073.439941,1056.780029,13528700.0,15722.910156,15644.969727,3.379850e+09,0.0,Friday,0
1,2021-12-30,46502.44,47207.99,998.412250,1061.329956,1070.339966,15680300.0,15758.980469,15741.559570,3.732730e+09,0.0,Thursday,2
2,2021-12-29,48017.93,46502.44,1119.628142,1098.640015,1086.189941,18718000.0,15794.919922,15766.219727,3.694500e+09,0.0,Wednesday,4
3,2021-12-28,49821.79,48017.93,1275.007551,1109.489990,1088.469971,20108000.0,15895.200195,15781.719727,3.623600e+09,0.0,Tuesday,3
4,2021-12-27,51036.12,49821.79,896.042079,1073.670044,1093.939941,23715300.0,15696.830078,15871.259766,3.730120e+09,0.0,Monday,1


In [12]:
!pip install keras==2.3.1
!pip install git+https://www.github.com/keras-team/keras-contrib.git

/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/wsuser/pip-req-build-2

In [32]:
import tensorflow
from tensorflow import keras
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input, LSTM
from tensorflow.keras.models import Model

In [132]:
batch_size = 32
timesteps = 1
epoch = 100

In [133]:
# Using the function defined in the LSTM course videos

def get_train_length(dataset, batch_size, test_percent):
    length = len(dataset)
    length *= 1 - test_percent
    train_length_values = []
    for x in range(int(length) - 100, int(length)):
        modulo = x % batch_size
        if (modulo == 0):
            train_length_values.append(x)
            print(x)
    return (max(train_length_values))

In [134]:
length = get_train_length(df, batch_size, 0.2)

1168
1184
1200
1216
1232
1248


In [135]:
df_train = df[0:length + timesteps*2]
df_test = df[length + timesteps*2 + 1:]

In [136]:
df_train = df_train[['TESLA Open', 'TESLA Volume', 'NASDAQ Open', 'NASDAQ Volume', 'Disaster', 'day', 'BTC Open']]

df_train_x = df_train[['TESLA Open', 'TESLA Volume', 'NASDAQ Open', 'NASDAQ Volume', 'Disaster', 'day']]
df_test_x = df_test[['TESLA Open', 'TESLA Volume', 'NASDAQ Open', 'NASDAQ Volume', 'Disaster', 'day']]

df_train_y = df_train['BTC Open']
df_test_y = df_test['BTC Open']

In [137]:
from sklearn.preprocessing import Normalizer

#norm = Normalizer(norm='max')
#df_train_x = norm.fit_transform(df_train_x)
df_train = np.array(df_train)

In [138]:
x_train = []
y_train = []

print(length+timesteps)
for i in range(timesteps, length + timesteps):
    x_train.append(df_train[i-timesteps:i,0:5])
    y_train.append(df_train[i:i+timesteps,6])
    
print(len(x_train))
print(len(y_train))

1249
1248
1248


In [139]:
import numpy as np
x_train, y_train = np.array(x_train), np.array(y_train)
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[2], 1))
y_train = np.reshape(y_train, (y_train.shape[0], y_train.shape[1], 1))

print(x_train.shape)
print(y_train.shape)

(1248, 5, 1)
(1248, 1, 1)


In [154]:
input_1 = Input(batch_shape=(32, 5, 1))
lstm_1 = LSTM(128, stateful=True, return_sequences=True)(input_1)
lstm_2 = LSTM(64, stateful=True, return_sequences=True)(lstm_1)
lstm_3 = LSTM(32, stateful=True, return_sequences=True)(lstm_2)
output_1 = Dense(units=1)(lstm_3)

regressor = Model(inputs=input_1, outputs=output_1)
regressor.compile(optimizer='adam', loss='mean_absolute_error')
regressor.summary()

Model: "model_20"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_25 (InputLayer)        [(32, 5, 1)]              0         
_________________________________________________________________
lstm_46 (LSTM)               (32, 5, 128)              66560     
_________________________________________________________________
lstm_47 (LSTM)               (32, 5, 64)               49408     
_________________________________________________________________
lstm_48 (LSTM)               (32, 5, 32)               12416     
_________________________________________________________________
dense_20 (Dense)             (32, 5, 1)                33        
Total params: 128,417
Trainable params: 128,417
Non-trainable params: 0
_________________________________________________________________


In [155]:
history = regressor.fit(
    x_train,
    y_train,
    epochs=8)

Epoch 1/8
39/39 [==============================] - 5s 67ms/step - loss: 15346.1125
Epoch 2/8
39/39 [==============================] - 3s 65ms/step - loss: 15711.2146
Epoch 3/8
39/39 [==============================] - 3s 68ms/step - loss: 15385.9822
Epoch 4/8
39/39 [==============================] - 3s 66ms/step - loss: 15293.9451
Epoch 5/8
39/39 [==============================] - 2s 64ms/step - loss: 15846.5281
Epoch 6/8
39/39 [==============================] - 3s 67ms/step - loss: 15562.8048
Epoch 7/8
39/39 [==============================] - 3s 65ms/step - loss: 15960.1273
Epoch 8/8
39/39 [==============================] - 2s 64ms/step - loss: 16065.9150
